# Tổng quan phần thu thập dữ liệu
1. Mục tiêu
- Hiện nay có rất nhiều nhà tuyển dụng đăng thông tin tuyển dụng trên nhiều trang web như : https://vieclam24h.vn/, https://www.topcv.vn/viec-lam, https://careerbuilder.vn/tim-viec-lam.html, https://www.vieclamtot.com/viec-lam
- Do đó để dự đoán xu hướng công việc cũng như tìm hiểu thị trường việc làm, chúng ta sẽ chọn thu thập dữ liệu từ các trang web [Viêc làm 24h](https://vieclam24h.vn/).
3. Công cụ sử dụng
- Trong phần này, ta sử dụng những thư viện được python hỗ trợ như: request_html để gửi yêu cầu lấy trang web về.
- BeautifulSoup để phân tích cấu trúc của trang web. Sau đó lấy những trường cần thiết.
5. Các bước thu thập dữ liệu
- Ban đầu trang chủ trang web có một các thông tin về các công việc mới nhất, các công việc được đăng gần đây nhất. Ta sẽ lấy các link của các công việc này. Sau đó đến đường dẫn các công việc này để lấy thông tin chi tiết về công việc. Chúng em sẽ có 2 bước thu thập dữ liệu:
    - Step 1: Thu thập dữ liệu từ trang chủ sau đó lưu các đường link vào 'job_pool.csv'
    - Thu thập dữ liệu từ các trang chi tiết và lưu tất cả công việc vào một dataset có tên là 'raw_dataset.csv'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import những  thư viện cần thiết
import pandas as pd
import numpy as np
!pip install requests_html
from requests_html import HTML
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import math
import requests
import time

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.6 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=1417d6155a76a6cef6caab8cbc55c4486ab049e96526614336a449b70cb0e533
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


## Hàm gửi yêu cầu đến server yêu cầu tải toàn bộ nội dung trang web

In [3]:

def get_content(url, headers):
    # time.sleep(1)
    session = HTMLSession()
    response = session.get(url, headers=headers)
    if(response.status_code == 200):
        # print("Success!!!")
        return response
    return None

In [4]:
BASE_URL = 'https://vieclam24h.vn'


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}


## Step 1: Lấy những liên kết của trang web công việc
- Đầu tiên, khi vô một trang web tìm việc, ta sẽ thấy một loạt danh sách các công việc, sau khi nhấn vào từng cái sẽ liên kết trực tiếp đến trang chi tiết công việc. Vậy đầu tiên ta đến trang tìm kiếm và lọc tất cả các đường dẫn đến công việc.

In [ ]:
# Code demo request lấy page list công việc
job_pool = pd.DataFrame()
for occupation_id in range(1, 54):
   for work_method in range(1,7):
      #https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=2&working_method[]=1
      url = BASE_URL + '/tim-kiem-viec-lam-nhanh?occupation_ids[]={}&working_method[]={}'.format(occupation_id,work_method)
      print(url)
      page_content = get_content(url, headers)
      try:
         soup = BeautifulSoup(page_content.text, 'html.parser')
         buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
         buffer2 = buffer1.find('span',class_="font-semibold").text
         n_jobs = int(buffer2.replace(',',''))
      except:
         n_jobs = 0
      # Trung bình mỗi page có 30 công việc
      n_pages = min(51, math.ceil(n_jobs/30) + 1)
      print('Có {} công việc, {} trang ở work_method = {} và occupation_id= {}'.format(n_jobs, n_pages, work_method, occupation_id))


      for page in range(1,n_pages):
         # Khởi tạo url vầ yêu cầu lấy page về
         url = BASE_URL + '/tim-kiem-viec-lam-nhanh?occupation_ids[]={}&page={}&working_method={}'.format(occupation_id,page, work_method)
         print(url)
         page_content = get_content(url, headers)
         try:
            soup = BeautifulSoup(page_content.text, 'html.parser')

            containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
         except:
            containers = []
         print('Đã tìm thấy {} công việc'.format(len(containers)))
         for container in containers:
            dict_job = {}

            buffer = BASE_URL + container.find('a', href=True)['href']
            dict_job['Liên kết'] = buffer if buffer else None

            buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
            dict_job['Tên công việc'] = buffer.text if buffer else None

            buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
            dict_job['Tên công ty'] = buffer.text if buffer else None

            buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
            dict_job['Mức lương'] = buffer.text if buffer else None

            buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
            dict_job['Khu vực tuyển'] = buffer.text if buffer else None

            job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool.sample(5)



https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&working_method[]=1
Có 603 công việc, 22 trang ở work_method = 1 và occupation_id= 1
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=1&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=2&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=3&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=4&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=5&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=6&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupation_ids[]=1&page=7&working_method=1
Đã tìm thấy 30 công việc
https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?occupat

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
6874,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Hỗ Trợ Thanh Toán Tại Địa Bàn_ ...,Công ty Tài Chính TNHH HD SAISON,15 - 30 triệu,Kiên Giang
2358,https://vieclam24h.vn/marketing/nhan-vien-mark...,Nhân Viên Marketing Chạy Quảng Cáo,Cty Cổ Phần Việt Quang Group,10 - 15 triệu,TP.HCM
3177,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Phát Triển Kinh Doanh,Công Ty TNHH Kamereo,7.5 - 20 triệu,"TP.HCM, Bình Dương"
365,https://vieclam24h.vn/hanh-chinh-thu-ky/nhan-v...,Nhân Viên Kế Toán Tổng Hợp,Công Ty TNHH DV Công Nghệ Số Dts,4 - 6 triệu,Đắk Lắk
9161,https://vieclam24h.vn/hanh-chinh-thu-ky/nhan-v...,Nhân Viên Quản Lý Sàn Thương Mại Điện Tử - Liv...,Công Ty TNHH Miracle Baby Việt Nam,12 - 15 triệu,TP.HCM


In [ ]:
# Lưu dữ liệu vào file csv
job_pool.drop_duplicates(inplace=True)
job_pool.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig', index=False)
# job_pool.to_csv('../data/job_pool.csv', encoding='utf-8-sig', index=False)

# Step 2: Thu thập dữ liệu của từng trang web
- Sau khi thu thập được các link trang web, ta tiến hành thu thập dữ liệu của từng trang web bằng cách sử dụng cách tương tự

In [5]:
# Đọc dữ liệu từ file job_links.csv
df = pd.read_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/job_pool.csv', encoding='utf-8-sig')
# df = pd.read_csv('../data/job_pool.csv')
df.sample(5)

,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
4715,https://vieclam24h.vn/y-te-cham-soc-suc-khoe/p...,Phó Phòng Nghiên Cứu Và Phát Triển Sản Phẩm,Công Ty Cổ Phần Dược Phẩm Medibest,20 - 40 triệu,Hà Nội
4634,https://vieclam24h.vn/ngan-hang/chuyen-gia-chu...,Chuyên Gia / Chuyên Viên Cao Cấp Đào Tạo Nghiệ...,Ngân Hàng TMCP Quốc Tế Việt Nam (VIB),15 - 45 triệu,TP.HCM
5060,https://vieclam24h.vn/xay-dung/nhan-vien-ky-th...,"Nhân Viên Kỹ Thuật Cơ Khí, Cửa, Nhôm Kính",Công Ty TNHH Central Home Việt Nam,9 - 13 triệu,Hà Nội
1796,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Nhân Viên Quản Lý Đơn Hàng,Công Ty TNHH Thương Mại Dịch Vụ Kỹ Thuật Điện ...,8 - 20 triệu,TP.HCM
2258,https://vieclam24h.vn/khoa-hoc-ky-thuat/nhan-v...,Nhân Viên Sửa Chữa Ô Tô,Saigonbus - Công Ty CP Xe Khách Sài Gòn,10 - 15 triệu,TP.HCM


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5591 entries, 0 to 5590
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Liên kết       5591 non-null   object
 1   Tên công việc  5591 non-null   object
 2   Tên công ty    5591 non-null   object
 3   Mức lương      5591 non-null   object
 4   Khu vực tuyển  5591 non-null   object
dtypes: object(5)
memory usage: 218.5+ KB


In [7]:
raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Thời gian thử việc','Cấp bậc', 'Yêu cầu giới tính','Số lượng tuyển', 'Hình thức làm việc','Độ tuổi', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

for i, url in enumerate(df['Liên kết']):
    print("Công việc thứ {}".format(i))
    job_content = get_content(url, headers=headers)
    if (job_content != None):
      soup = BeautifulSoup(job_content.text, 'html.parser')

    dict_job = {}

    dict_job['Liên kết'] = url

    overview = soup.find('div', class_='md:ml-7 w-full')
    try:
        buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
        dict_job['Tên công ty'] = buffer.text if buffer else None
    except:
        dict_job['Tên công ty'] = None
    try:
        buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
        dict_job['Tên công việc'] = buffer.text if buffer else None
    except:
        dict_job['Tên công việc'] = None
    try:
        buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
        dict_job['Mức lương'] = buffer.text if buffer else None
    except:
        dict_job['Mức lương'] = None

    try:
        buffer = overview.find_all('a', class_ ="hover:text-se-accent")
        dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
    except:
        dict_job['Khu vực tuyển'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu giới tính")
        dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu giới tính'] = None

    try:
        buffer = soup.find('p', string="Cấp bậc")
        dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Cấp bậc'] = None

    try:
        buffer = soup.find('p', string="Thời gian thử việc")
        dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Thời gian thử việc'] = None

    try:
        buffer = soup.find('p', string="Số lượng tuyển")
        dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Số lượng tuyển'] = None


    try:
        buffer = soup.find('p', string="Hình thức làm việc")
        dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Hình thức làm việc'] = None

    try:
        buffer = soup.find('p', string="Độ tuổi")
        dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Độ tuổi'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu bằng cấp")
        dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu bằng cấp'] = None

    try:
        buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
        dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Yêu cầu kinh nghiệm'] = None

    try:
        buffer = soup.find('p', string="Ngành nghề")
        dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
    except:
        dict_job['Ngành nghề'] = None

    try:
        buffer1 = soup.find('h4', string="Mô tả công việc")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None
    except:
        dict_job['Mô tả công việc'] = None

    try:
        buffer1 = soup.find('h4', string="Yêu cầu công việc")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None
    except:
        dict_job['Yêu cầu công việc'] = None

    try:
        buffer1 = soup.find('h4', string="Quyền lợi")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Quyền lợi'] = buffer2.text if buffer2 else None
    except:
        dict_job['Quyền lợi'] = None

    try:
        buffer1 = soup.find('h4', string="Địa điểm làm việc")
        buffer2 = buffer1.find_next_sibling('ul')
        buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
        dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None
    except:
        dict_job['Địa điểm làm việc'] = None

    try:
        buffer1 = soup.find('h3', string="Từ khoá")
        buffer2 = buffer1.find_next_sibling('div')
        dict_job['Từ khóa'] = buffer2.text if buffer2 else None
    except:
        dict_job['Từ khóa'] = None

    company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

    buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
    try:
        for buffer in buffers:
            if buffer.get_text() == 'Địa chỉ:':
                dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
            if buffer.get_text() == 'Quy mô:':
                dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
    except:
        dict_job['Địa chỉ công ty'] = None
        dict_job['Quy mô công ty'] = None

    raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
    raw_job_details

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Công việc thứ 591
Công việc thứ 592
Công việc thứ 593
Công việc thứ 594
Công việc thứ 595
Công việc thứ 596
Công việc thứ 597
Công việc thứ 598
Công việc thứ 599
Công việc thứ 600
Công việc thứ 601
Công việc thứ 602
Công việc thứ 603
Công việc thứ 604
Công việc thứ 605
Công việc thứ 606
Công việc thứ 607
Công việc thứ 608
Công việc thứ 609
Công việc thứ 610
Công việc thứ 611
Công việc thứ 612
Công việc thứ 613
Công việc thứ 614
Công việc thứ 615
Công việc thứ 616
Công việc thứ 617
Công việc thứ 618
Công việc thứ 619
Công việc thứ 620
Công việc thứ 621
Công việc thứ 622
Công việc thứ 623
Công việc thứ 624
Công việc thứ 625
Công việc thứ 626
Công việc thứ 627
Công việc thứ 628
Công việc thứ 629
Công việc thứ 630
Công việc thứ 631
Công việc thứ 632
Công việc thứ 633
Công việc thứ 634
Công việc thứ 635
Công việc thứ 636
Công việc thứ 637
Công việc thứ 638
Công việc thứ 639
Công việc thứ 640
Công việc thứ 641
Công việc thứ 642
Công vi

In [8]:
raw_job_details.sample(5)

,Liên kết,Tên công ty,Tên công việc,Mức lương,Khu vực tuyển,Thời gian thử việc,Cấp bậc,Yêu cầu giới tính,Số lượng tuyển,Hình thức làm việc,...,Yêu cầu bằng cấp,Yêu cầu kinh nghiệm,Ngành nghề,Địa điểm làm việc,Mô tả công việc,Yêu cầu công việc,Quyền lợi,Từ khóa,Địa chỉ công ty,Quy mô công ty
2962,https://vieclam24h.vn/marketing/nhan-vien-mark...,Công Ty TNHH TiPi Global,Nhân Viên Marketing,8 - 9 triệu,TP.HCM,2 tháng,Chuyên viên- nhân viên,Nữ,1,Toàn thời gian cố định,...,Trung cấp,Dưới 1 năm,Marketing,"TP.HCM, 45 Hà Huy Tập, Phường Tân Phong, Quận 7",Thời gian làm việc: 8h30AM-17h30PM từ Thứ 2 đế...,Ưu tiên ứng viên nữ.Có am hiểu về nền tảng Tik...,Lương từ 8.000.000 VNĐ – 9.000.000 VNĐ (có thể...,nhân viên marketing tại hà nộilập trình unityv...,"45 Hà Huy Tập, Phú Mỹ Hưng, Tân Phong, Quận 7,...",10 - 150 người
3601,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty TNHH Pnk,Nhân Viên Mua Hàng Quốc Tế (Tiếng Trung),12 - 15 triệu,Hà Nội,2 tháng,Chuyên viên- nhân viên,None,2,Toàn thời gian cố định,...,Đại học,1 năm,Bán hàng - Kinh doanh/Xuất Nhập Khẩu/Thu mua -...,"Hà Nội, Số 14, Kim Đồng, Hoàng Mai","Tìm kiếm hàng hóa, nhà cung cấp nước ngoài;Soạ...","Nam, nữ từ 23-35 tuổi, Được đào tạo bài bản ti...",Lương khởi điểm: 12 đến 15 triệuĐược Chủ tịch ...,trungmua hàngtiếng hoa tại tphcmViệc làm Hà Nộ...,"Số 14 Kim Đồng, Giáp Bát, Hoàng Mai, Hà Nội",Trên 300 người
271,https://vieclam24h.vn/hanh-chinh-thu-ky/tro-ly...,Công Ty Cổ Phần Đầu Tư Và Quảng Cáo Century,Trợ Lý Giám Đốc,8 - 15 triệu,TP.HCM,None,Chuyên viên- nhân viên,Nữ,1,Toàn thời gian cố định,...,Đại học,1 năm,Hành chính - Thư ký,"TP.HCM, 816/64/28 Quốc lộ 1, thạnh xuân, Quận 12",- Trợ giúp Giám đốc trong việc hoạch định triể...,- Ưu tiên ứng viên tốt nghiệp Đại học các chuy...,- Thu nhập: lương + Phụ cấp + Thưởng trao đổi ...,quận 12 thành phố hồ chí minhViệc làm TP....,"816/64/28 Quốc lộ 1, thạnh xuân, quận 12, Tp.H...",Dưới 10 người
1867,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty TNHH Minh Mẫn Mê Kông,Nhân Viên Kinh Doanh,10 - 15 triệu,Long An,1 tháng,Chuyên viên- nhân viên,Nam,3,Toàn thời gian cố định,...,Cao đẳng,1 năm,Bán hàng - Kinh doanh/Bán sỉ - Bán lẻ - Quản l...,"Long An, Gần cầu Đức Hòa, Đức Hòa","Tìm kiếm khách hàng.Chăm sóc khách hàng, giải ...","Nhanh nhẹn, cẩn thận, trung thực, có trách nhi...","THưởng đầy đủ BHXH, BHYT, phép năm.... sau thờ...",huyện bến lứclập trình unityvisual merchandisi...,"Số Thửa 369 ấp 1, xã Thạnh Lợi, huyện Bến Lức,...",10 - 150 người
2823,https://vieclam24h.vn/ban-hang-kinh-doanh/nhan...,Công Ty Cổ Phần Win Win Corp,Nhân Viên Kinh Doanh Thị Trường,8 - 25 triệu,TP.HCM,2 tháng,Chuyên viên- nhân viên,None,5,Toàn thời gian cố định,...,None,Dưới 1 năm,Bán hàng - Kinh doanh/Chăm sóc khách hàng/Mark...,"TP.HCM, 144, Đường 1, Long Trường, Thủ Đức",Tìm kiếm khách hàng mới và phát triển mối quan...,"Thái độ làm việc tốt: tuân thủ nội quy, văn hó...",Thu nhập hấp dẫnThưởng lương tháng 13.Công ty ...,thị trườngthành phố hải dươngkinh doanh thị tr...,"59 Nguyễn Đình Bể, Phường Tân Bình, Thành phố ...",10 - 150 người


In [9]:
# Lưu vào file craw_dataset.csv
raw_job_details.info()
# raw_job_details.to_csv('../data/raw_dataset.csv', encoding='utf-8-sig', index=False)
raw_job_details.to_csv('/content/drive/MyDrive/HCMUS/HK5/Intro2DS/Final Project/data/raw_dataset.csv', encoding='utf-8-sig', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5591 entries, 0 to 5590
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Liên kết             5591 non-null   object
 1   Tên công ty          5591 non-null   object
 2   Tên công việc        5591 non-null   object
 3   Mức lương            5591 non-null   object
 4   Khu vực tuyển        5591 non-null   object
 5   Thời gian thử việc   3577 non-null   object
 6   Cấp bậc              5591 non-null   object
 7   Yêu cầu giới tính    1927 non-null   object
 8   Số lượng tuyển       5591 non-null   object
 9   Hình thức làm việc   5591 non-null   object
 10  Độ tuổi              2817 non-null   object
 11  Yêu cầu bằng cấp     4905 non-null   object
 12  Yêu cầu kinh nghiệm  5591 non-null   object
 13  Ngành nghề           5591 non-null   object
 14  Địa điểm làm việc    5581 non-null   object
 15  Mô tả công việc      5591 non-null   object
 16  Yêu cầ

# Một số phần demo để testing
- Đây là đoạn code để người collect data có thể test thử trước khi chạy toàn bộ code

In [ ]:
# Demo lất page content
url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text
soup = BeautifulSoup(page_text, 'html.parser')
buffer1 = soup.find('div', class_='relative lg:text-2xl text-xl box-border lg:leading-10 mb-4 m-auto lg:w-full w-[100%] px-0')
buffer2 = buffer1.find('span',class_="font-semibold").text
n_jobs = int(buffer2.replace(',',''))
print('Có {} công việc'.format(n_jobs))
n_pages = min(51, math.ceil(n_jobs/30) + 1)
print('Có {} trang'.format(n_pages-1))

Success!!!
Có 101 công việc
Có 4 trang


In [ ]:
# Demo lất page content

url = 'https://vieclam24h.vn/tim-kiem-viec-lam-nhanh?page=2&working_method=6'
page_content = get_content(url, headers)
page_text = page_content.text

job_pool = pd.DataFrame(columns=['Liên kết', 'Tên công việc','Tên công ty','Mức lương','Khu vực tuyển'])

soup = BeautifulSoup(page_text, 'html.parser')
containers = soup.find_all('div', class_='relative lg:h-[115px] w-full flex rounded-sm lg:mb-3 mb-2 lg:hover:shadow-md')
print('Đã tìm thấy {} công việc'.format(len(containers)))
for container in containers:
    dict_job = {}

    buffer = BASE_URL + container.find('a', href=True)['href']
    dict_job['Liên kết'] = buffer if buffer else None

    buffer = container.find('div', class_='relative lg:w-full w-11/12 flex items-start flex-1 overflow-hidden pr-2 lg:pr-8')
    dict_job['Tên công việc'] = buffer.text if buffer else None

    buffer = container.find('p', class_='inline-block text-grey-48 text-[16px] leading-6 truncate pr-2 max-w-[240px] lg:max-w-full')
    dict_job['Tên công ty'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 text-14 whitespace-nowrap font-medium')
    dict_job['Mức lương'] = buffer.text if buffer else None

    buffer = container.find('span', class_='text-se-neutral-80 whitespace-nowrap text-14')
    dict_job['Khu vực tuyển'] = buffer.text if buffer else None

    job_pool = pd.concat([job_pool, pd.DataFrame(dict_job, index=[0])], ignore_index=True)

job_pool

Success!!!
Đã tìm thấy 30 công việc


,Liên kết,Tên công việc,Tên công ty,Mức lương,Khu vực tuyển
0,https://vieclam24h.vn/bien-phien-dich/thuc-tap...,Thực Tập Sinh Biên Dịch Tiếng Anh - Lương 5Tr ...,Công Ty TNHH Mediastep Software Việt Nam,5 - 6 triệu,TP.HCM
1,https://vieclam24h.vn/thiet-ke-sang-tao-nghe-t...,Thực Tập Sinh Thiết Kế Đồ Họa,Công Ty TNHH Miligo,1 - 5 triệu,TP.HCM
2,https://vieclam24h.vn/bat-dong-san/thuc-tap-si...,Thực Tập Sinh Kinh Doanh Tại Thanh Hóa,Công Ty Cp Đầu Tư Và Kinh Doanh Bất Động Sản S...,5 - 10 triệu,Thanh Hóa
3,https://vieclam24h.vn/nhan-su/thuc-tap-sinh-nh...,Thực Tập Sinh Nhân Sự (C&B),Công Ty CP Pt Kinh Doanh Thịnh Phát,2.5 - 3 triệu,Hà Nội
4,https://vieclam24h.vn/marketing/thuc-tap-sinh-...,Thực Tập Sinh Marketing,Công Ty Cổ Phần Khoa Học Công Nghệ Xanh,1 - 3 triệu,Hà Nội
5,https://vieclam24h.vn/it-phan-mem/thuc-tap-sin...,Thực Tập Sinh Lập Trình PHP,Công Ty Cổ Phần Vinasimex,10 - 30 triệu,"Hà Nội, TP.HCM"
6,https://vieclam24h.vn/khach-san-nha-hang-du-li...,Thực Tập Sinh Kế Toán,Công Ty TNHH Sôcôla Marou,1 - 3 triệu,TP.HCM
7,https://vieclam24h.vn/ban-hang-kinh-doanh/thuc...,Thực Tập Sinh Trình Dược Viên,Công Ty Cổ Phần Đất Dược,2 - 9 triệu,"TP.HCM, Bình Dương, Cần Thơ, Đồng Nai"
8,https://vieclam24h.vn/ban-hang-kinh-doanh/q1-h...,[Q1 - HCM] Thực Tập Sinh Phòng Kinh Doanh B2B,Cty TNHH Phòng Khám Đa Khoa Jio Health,3 - 3.5 triệu,TP.HCM
9,https://vieclam24h.vn/thuc-tap-sinh/thuc-tap-s...,Thực Tập Sinh Kinh Doanh Thiết Bị Ngành Kỹ Thuật,Công Ty Cổ Phần Tự Động Việt,2 - 8 triệu,TP.HCM


In [ ]:

url = 'https://vieclam24h.vn/ban-hang/cong-tac-vien-ban-hang-online-my-pham-thuc-pham-chuc-nang-c113p121id200062770.html'
job_content = get_content(url, headers=headers).text
soup = BeautifulSoup(job_content, 'html.parser')

Success!!!


In [ ]:
# Demo lấy chi tiết

raw_job_details = pd.DataFrame(columns=['Liên kết','Tên công ty', 'Tên công việc','Mức lương','Khu vực tuyển',
# Thông tin chung
'Thời gian thử việc','Cấp bậc', 'Yêu cầu giới tính','Số lượng tuyển', 'Hình thức làm việc','Độ tuổi', 'Yêu cầu bằng cấp', 'Yêu cầu kinh nghiệm', 'Ngành nghề', 'Địa điểm làm việc',
'Mô tả công việc', 'Yêu cầu công việc', 'Quyền lợi', 'Từ khóa',
# Thông tin về công ty
'Địa chỉ công ty', 'Quy mô công ty'])

dict_job = {}

dict_job['Liên kết'] = url

overview = soup.find('div', class_='md:ml-7 w-full')
try:
    buffer = overview.find('h3', class_='font-normal text-16 text-se-neutral-64 mb-4')
    dict_job['Tên công ty'] = buffer.text if buffer else None
except:
    dict_job['Tên công ty'] = None
try:
    buffer = overview.find('h1', class_='font-semibold text-18 md:text-24 leading-snug')
    dict_job['Tên công việc'] = buffer.text if buffer else None
except:
    dict_job['Tên công việc'] = None
try:
    buffer = overview.find('p', class_='font-semibold text-14 text-[#8B5CF6]')
    dict_job['Mức lương'] = buffer.text if buffer else None
except:
    dict_job['Mức lương'] = None

try:
    buffer = overview.find_all('a', class_ ="hover:text-se-accent")
    dict_job['Khu vực tuyển'] = ' '.join([element.text for element in buffer]) if buffer else None
except:
    dict_job['Khu vực tuyển'] = None

try:
    buffer = soup.find('p', string="Yêu cầu giới tính")
    dict_job['Yêu cầu giới tính'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu giới tính'] = None

try:
    buffer = soup.find('p', string="Cấp bậc")
    dict_job['Cấp bậc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Cấp bậc'] = None

try:
    buffer = soup.find('p', string="Thời gian thử việc")
    dict_job['Thời gian thử việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Thời gian thử việc'] = None

try:
    buffer = soup.find('p', string="Số lượng tuyển")
    dict_job['Số lượng tuyển'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Số lượng tuyển'] = None


try:
    buffer = soup.find('p', string="Hình thức làm việc")
    dict_job['Hình thức làm việc'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Hình thức làm việc'] = None

try:
    buffer = soup.find('p', string="Độ tuổi")
    dict_job['Độ tuổi'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Độ tuổi'] = None

try:
    buffer = soup.find('p', string="Yêu cầu bằng cấp")
    dict_job['Yêu cầu bằng cấp'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu bằng cấp'] = None

try:
    buffer = soup.find('p', string="Yêu cầu kinh nghiệm")
    dict_job['Yêu cầu kinh nghiệm'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Yêu cầu kinh nghiệm'] = None

try:
    buffer = soup.find('p', string="Ngành nghề")
    dict_job['Ngành nghề'] = buffer.find_next_sibling('p').get_text() if buffer else None
except:
    dict_job['Ngành nghề'] = None

try:
    buffer1 = soup.find('h4', string="Mô tả công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Mô tả công việc'] = buffer2.text if buffer2 else None
except:
    dict_job['Mô tả công việc'] = None

try:
    buffer1 = soup.find('h4', string="Yêu cầu công việc")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Yêu cầu công việc'] = buffer2.text if buffer2 else None
except:
    dict_job['Yêu cầu công việc'] = None

try:
    buffer1 = soup.find('h4', string="Quyền lợi")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Quyền lợi'] = buffer2.text if buffer2 else None
except:
    dict_job['Quyền lợi'] = None

try:
    buffer1 = soup.find('h4', string="Địa điểm làm việc")
    buffer2 = buffer1.find_next_sibling('ul')
    buffer3 = buffer2.find_all('li', class_ = "jsx-d84db6a84feb175e mb-2 flex text-14")
    dict_job['Địa điểm làm việc'] = '; '.join([element.text for element in buffer3 if element.text.strip()]) if buffer3 else None
except:
    dict_job['Địa điểm làm việc'] = None

try:
    buffer1 = soup.find('h3', string="Từ khoá")
    buffer2 = buffer1.find_next_sibling('div')
    dict_job['Từ khóa'] = buffer2.text if buffer2 else None
except:
    dict_job['Từ khóa'] = None

company = soup.find('div', class_='px-4 md:px-10 py-4 bg-white shadow-sd-12 rounded-sm mt-4 lg:mb-6')

buffers = company.find_all('div', class_= 'text-14 text-se-neutral-64 w-full max-w-[125px] mr-2')
try:
    for buffer in buffers:
        if buffer.get_text() == 'Địa chỉ:':
            dict_job['Địa chỉ công ty'] = buffer.find_next_sibling('div').text if buffer else None
        if buffer.get_text() == 'Quy mô:':
            dict_job['Quy mô công ty'] = buffer.find_next_sibling('div').text if buffer else None
except:
    dict_job['Địa chỉ công ty'] = None
    dict_job['Quy mô công ty'] = None

raw_job_details = pd.concat([raw_job_details, pd.DataFrame(dict_job, index=[0])], ignore_index=True)
raw_job_details


0    25 - 29 tuổi
Name: Độ tuổi, dtype: object